In [1]:
import requests
import folium
import polyline
import pandas as pd
import geopandas as gpd

In [2]:
rs = pd.read_csv("Wilkerstat_RumahSakit.csv")
grid = gpd.read_file('Variabel X.geojson')

In [3]:
def get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):
    
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://router.project-osrm.org/route/v1/driving/"
    addition = "alternatives={false}&steps={true}&overview={full}&annotations={false}"
    r = requests.get(url + loc) 
    
    if r.status_code!= 200:
        return {}
  
    res = r.json()   
    routes = polyline.decode(res['routes'][0]['geometry'])
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    distance = res['routes'][0]['distance']
    duration = res['routes'][0]['duration']
    
    out = {
#         'full':res,
        'route':routes,
           'start_point':start_point,
           'end_point':end_point,
           'distance':distance,
        'duration':duration
          }

    return out

In [4]:
grid = grid.drop(['WILKERSTAT_POI Density Persalinan', 'WILKERSTAT_POI Density Imunisasi',
       'WILKERSTAT_POI Density KB Modern', 'WILKERSTAT_POI Density PAUD',
       'WILKERSTAT_POI Density Ekonomi', 'WILKERSTAT_POI Distance Imunisasi',
       'WILKERSTAT_POI Distance KB Modern', 'WILKERSTAT_POI Distance PAUD',
       'WILKERSTAT_POI Distance Persalinan', 'OSM_POI Distance KB Modern',
       'OSM_POI Density KB Modern', 'OSM_POI Distance PAUD',
       'OSM_POI Density PAUD', 'OSM_POI Distance Persalinan dan Imunisasi',
       'OSM_POI Density Persalinan dan Imunisasi'],1)

C:\Users\fauza\AppData\Local\Temp/ipykernel_11612/3265384922.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  grid = grid.drop(['WILKERSTAT_POI Density Persalinan', 'WILKERSTAT_POI Density Imunisasi',


In [5]:
grid["lon"] = (grid["right"] + grid["left"])/2
grid["lat"] = (grid["top"] + grid["bottom"])/2

In [6]:
rs

,Unnamed: 0,Unnamed: 0.1,nama,deskripsi,latitude,longitude,accuracy,kode_kat,iddesa,no,kode,jenis
0,0,279,Warung nasi TO Ibu Engkay,Kp. Rahayu Rt 009 Rw 004,-7.374405,108.154103,4.551,30020001,3206191003,21,2,Rumah sakit
1,1,280,Mesjid Jamie Persatuan Islam,Depan Lapangan Bola,-7.702637,108.038752,1.600,30020001,3206010008,21,2,Rumah sakit
2,2,281,TKQ Annur,bangunan pendidikan,-6.806397,107.146266,3.216,30020001,3203200005,21,2,Rumah sakit
3,3,282,rumah sakit,rumah sakit banjar patroman dekat waterpark,-7.361761,108.535298,3.216,30020001,3279010005,21,2,Rumah sakit
4,4,283,Rumah Sakit,Rt. 01 Rw. 01,-6.482908,106.870340,5.400,30020001,3201200014,21,2,Rumah sakit
...,...,...,...,...,...,...,...,...,...,...,...,...
856,856,288767,KLINIK DEWI ADHI SUCI,Rt 001 Rw 001,-6.270757,107.113375,3.216,30020002,3216071010,22,2,Rumah sakit bersalin
857,857,288768,klinik nyi nita,kesehat,-6.333665,107.058868,3.200,30020002,3216071001,22,2,Rumah sakit bersalin
858,858,290159,praktek bidan dewi,praktek bidan,-6.928755,107.122137,3.216,30020002,3203120003,22,2,Rumah sakit bersalin
859,859,292913,rumah sakit,rumah sakit swasta,-6.886651,107.611683,3.000,30020002,3273230002,22,2,Rumah sakit bersalin


In [ ]:
hasil = {
    'pickup_id':[],
    'dropoff_id':[],
    'start_point':[],
    'end_point':[],
    'distance':[],
    'duration':[],
    'routes':[],
    
}
for i in range(grid.shape[0]):
    print(i)
    for j in range(rs.shape[0]):
        g = grid.loc[i]
        r = rs.loc[j]
        pickup_lon, pickup_lat, dropoff_lon, dropoff_lat = g['lon'],g['lat'],r['longitude'],r['latitude']
        test_route = get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
#         print(test_route)
        hasil["pickup_id"].append(g["id"])
        hasil["dropoff_id"].append(r["Unnamed: 0"])
        hasil["start_point"].append(test_route["start_point"])
        hasil["end_point"].append(test_route["end_point"])
        hasil["distance"].append(test_route["distance"])
        hasil["duration"].append(test_route["duration"])
        hasil["routes"].append(str(test_route["route"]))
        

0
1
2
3
4
5


In [ ]:
pd.DataFrame(hasil).set_index('pickup_id').sort_values('distance').groupby('pickup_id').min()

In [ ]:
pd.DataFrame(hasil).set_index('pickup_id').sort_values('distance').groupby('pickup_id').min().to_csv('OSRM_Wilkerstat_RS.csv')